In [1]:
!pip install obonet -q
!pip install pyvis -q

In [2]:
import os
import json
from PIL import Image
from typing import Dict
from collections import Counter

import random
import cv2
import obonet
import networkx
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.patches as mpatch
from Bio import SeqIO
from pyvis.network import Network

In [3]:
class CFG:
    train_go_obo_path: str = "/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo"
    train_seq_fasta_path: str = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta"
    train_terms_path: str = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv"
    train_taxonomy_path: str = "/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv"
    train_ia_path: str = "/kaggle/input/cafa-5-protein-function-prediction/IA.txt"

In [4]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [5]:
def plot_dag(graph, term, radius=1):
    # create smaller subgraph
    # radius - include all neighbors of distance<=radius from n (increse it to add further parent's branches).
    ng_graph = networkx.ego_graph(graph, term, radius=radius)

    for n in ng_graph.nodes(data=True):
        # concatenate label of the node with its attribute
        n[1]["label"] = n[0] + " " +n[1]["name"]

    nt = Network(directed=True, notebook=True, cdn_resources="in_line")
    nt.from_nx(ng_graph)
    return nt.show("network.html")

In [6]:
subontology_roots = {'BPO':'GO:0008150',
                     'CCO':'GO:0005575',
                     'MFO':'GO:0003674'}

In [7]:
%%time
graph = obonet.read_obo(CFG.train_go_obo_path)

CPU times: user 7.72 s, sys: 103 ms, total: 7.82 s
Wall time: 7.84 s


In [8]:
print(f"Number of edges: {graph.number_of_edges()}")

Number of edges: 84805


In [9]:
print(f"Number of nodes: {len(graph)}")

Number of nodes: 43248


In [10]:
term = "GO:0034655"

In [11]:
graph.nodes[term]

{'name': 'nucleobase-containing compound catabolic process',
 'namespace': 'biological_process',
 'def': '"The chemical reactions and pathways resulting in the breakdown of nucleobases, nucleosides, nucleotides and nucleic acids." [GOC:mah]',
 'subset': ['goslim_chembl'],
 'synonym': ['"nucleobase, nucleoside, nucleotide and nucleic acid breakdown" EXACT []',
  '"nucleobase, nucleoside, nucleotide and nucleic acid catabolic process" RELATED [GOC:dph, GOC:tb]',
  '"nucleobase, nucleoside, nucleotide and nucleic acid catabolism" EXACT []',
  '"nucleobase, nucleoside, nucleotide and nucleic acid degradation" EXACT []'],
 'is_a': ['GO:0006139',
  'GO:0019439',
  'GO:0044270',
  'GO:0046700',
  'GO:1901361']}

In [12]:
plot_dag(graph, term, radius=1)

network.html


In [13]:
plot_dag(graph, term, radius=1000)

network.html


In [14]:
term = "GO:0044270"

In [15]:
graph.nodes[term]

{'name': 'cellular nitrogen compound catabolic process',
 'namespace': 'biological_process',
 'def': '"The chemical reactions and pathways resulting in the breakdown of organic and inorganic nitrogenous compounds." [GOC:jl, ISBN:0198506732]',
 'synonym': ['"nitrogen compound breakdown" BROAD []',
  '"nitrogen compound catabolism" BROAD []',
  '"nitrogen compound degradation" BROAD []'],
 'is_a': ['GO:0034641', 'GO:0044248']}

In [16]:
plot_dag(graph, term, radius=1)

network.html


In [17]:
plot_dag(graph, term, radius=1000)

network.html


In [18]:
print("Sequence example:\n\n", next(iter(SeqIO.parse(CFG.train_seq_fasta_path, "fasta"))))

Sequence example:

 ID: P20536
Name: P20536
Description: P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosylase OS=Vaccinia virus (strain Copenhagen) OX=10249 GN=UNG PE=1 SV=1
Number of features: 0
Seq('MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIPDKFFIQLK...FIY')


In [19]:
sequences = SeqIO.parse(CFG.train_seq_fasta_path, "fasta")
num_sequences = sum(1 for seq in sequences)

print("Number of sequences:", num_sequences)

Number of sequences: 142246


In [20]:
sequences = SeqIO.parse(CFG.train_seq_fasta_path, "fasta")
avg_length = sum(len(seq) for seq in sequences)/num_sequences
avg_length

553.6366787115279

In [21]:
sequences = SeqIO.parse(CFG.train_seq_fasta_path, "fasta")
lengths = [len(seq) for seq in sequences]
median = np.sort(lengths)
median[int(num_sequences/2)]

411

In [22]:
sequences = SeqIO.parse(CFG.train_seq_fasta_path, "fasta")

# get the length of each sequence
lengths = [len(seq) for seq in sequences]

fig = px.histogram(x=lengths, nbins=1000, color_discrete_sequence=['goldenrod'],histfunc = "min")
fig.update_layout(
    title={
        'text': "Distribution of protein sequence lengths",
        'y':0.95,
        'x':1.0,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title="Sequence length", yaxis_title="Count"
)

fig.show()

In [23]:
np.percentile(lengths, 90)

1054.0

In [24]:
taxon_df = pd.read_csv(CFG.train_taxonomy_path,'\t')
taxon_df.head()

/tmp/ipykernel_20/3004351233.py:1: FutureWarning:

In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.



,EntryID,taxonomyID
0,Q8IXT2,9606
1,Q04418,559292
2,A8DYA3,7227
3,Q9UUI3,284812
4,Q57ZS4,185431


In [25]:
terms_df = pd.read_csv(CFG.train_terms_path,'\t')
terms_df.head()

/tmp/ipykernel_20/1404086741.py:1: FutureWarning:

In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.



,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO


In [26]:
num_GO_terms = len(pd.unique(terms_df['term']))
num_protein_terms = len(pd.unique(terms_df['EntryID']))
print(f"Number of Unique Proteins: {num_protein_terms}")
print(f"Number of Unique GO terms: {num_GO_terms}")

Number of Unique Proteins: 142246
Number of Unique GO terms: 31466


In [27]:
import pandas as pd
import numpy as np

def stat_calcs(df, colname):
    if 'Ontology' in df.columns:
        all_ont = pd.DataFrame()
        for ont in df['Ontology'].unique():
            df_ont = df[df['Ontology'] == ont]
            out = df_ont.agg({
                colname: ['min', lambda x: np.percentile(x, 10), lambda x: np.percentile(x, 25),
                          'median', 'mean', lambda x: np.percentile(x, 75),
                          lambda x: np.percentile(x, 90), 'max']
            }).T.reset_index().rename(columns={'index': 'stat'})
            out['Ontology'] = ont
            all_ont = all_ont.append(out)
        
        out = all_ont.melt(id_vars=['stat'], value_vars=['Ontology'], var_name='Ontology') \
            .pivot(index='stat', columns='Ontology', values='value') \
            .apply(lambda x: round(x, 0) if pd.api.types.is_numeric_dtype(x) else x)
        
    else:
        out = df.agg({
            colname: ['min', lambda x: np.percentile(x, 10), lambda x: np.percentile(x, 25),
                      'median', 'mean', lambda x: np.percentile(x, 75),
                      lambda x: np.percentile(x, 90), 'max']
        }).T.reset_index().rename(columns={'index': 'stat'})
        out[colname] = out[colname].round(0)
    
    out['stat'] = pd.Categorical(out['stat'], categories=out['stat'])
    return out

In [28]:
# Number of terms per protein
dstats_prot = terms_df.groupby('EntryID').agg(term_per_prot=('term', 'nunique')).reset_index()

# Number of proteins per GO term
dstats_term = terms_df.groupby('term').agg(prot_per_term=('EntryID', 'nunique')).reset_index()

In [29]:
dstats_term.head()

,term,prot_per_term
0,GO:0000001,26
1,GO:0000002,133
2,GO:0000003,8499
3,GO:0000006,3
4,GO:0000007,1


In [30]:
dstats_prot.head()

,EntryID,term_per_prot
0,A0A009IHW8,49
1,A0A021WW32,77
2,A0A021WZA4,5
3,A0A023FBW4,6
4,A0A023FBW7,6


In [31]:
colname = 'term_per_prot'
out = dstats_prot[colname].agg(['min',
                                lambda x: x.quantile(0.1),
                                lambda x: x.quantile(0.25),
                                'median',
                                'mean',
                                lambda x: x.quantile(0.75),
                                lambda x: x.quantile(0.90),
                                'max']).reset_index()

out.columns = ['stat', colname]
out[colname] = out[colname].round(0)
out = out.drop_duplicates(subset='stat')  # Remove duplicates in 'stat' column
out['stat'] = pd.Categorical(out['stat'], categories=out['stat'], ordered=True)


In [32]:
out.head()

,stat,term_per_prot
0,min,2.0
1,<lambda>,5.0
3,median,24.0
4,mean,38.0
7,max,815.0


In [33]:
fig = px.histogram(x=dstats_prot['term_per_prot'], nbins=200, color_discrete_sequence=['red'])
fig.update_layout(
    title={
        'text': "Distribution of GO term per protein",
        #'y':0.95,
        #'x':0.5,
        #'xanchor': 'center',
        #'yanchor': 'top'
    },
    xaxis_title="Number of terms per protein", yaxis_title="Count"
)

fig.show()

In [34]:
fig = px.histogram(x=dstats_term['prot_per_term'], nbins=5, color_discrete_sequence=['green'])
fig.update_layout(
    title={
        'text': "Distribution of protein per GO term",
        #'y':0.95,
        #'x':0.5,
        #'xanchor': 'center',
        #'yanchor': 'top'
    },
    xaxis_title="Number of proteins per GO term", yaxis_title="Count"
)

fig.show()

In [35]:
colname = 'prot_per_term'
out2 = dstats_term[colname].agg(['min',
                                lambda x: x.quantile(0.1),
                                lambda x: x.quantile(0.25),
                                'median',
                                'mean',
                                lambda x: x.quantile(0.75),
                                lambda x: x.quantile(0.90),
                                'max']).reset_index()

out2.columns = ['stat', colname]
out2[colname] = out2[colname].round(0)
out2 = out2.drop_duplicates(subset='stat')  # Remove duplicates in 'stat' column
out2['stat'] = pd.Categorical(out2['stat'], categories=out2['stat'], ordered=True)
out2.head()

,stat,prot_per_term
0,min,1.0
1,<lambda>,1.0
3,median,8.0
4,mean,170.0
7,max,92912.0
